In [1]:
import torch
from torch.utils.data import Dataset
import h5py
import json
import os
import pandas as pd
import random

In [2]:
df_path='/home/ss4yd/vision_transformer/captioning_vision_transformer/prepared_prelim_data_tokenized.pickle'

In [3]:
pd.read_pickle(df_path).head()

,pid,tissue_type,notes,svs_path,patch_path,reps_path,tokens,p_tokens,caplens,dtype,idx_tokens
0,GTEX-1117F-0726,Heart - Atrial Appendage,"2 pieces, no abnormalities",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, no, abnormalities]","[<start>, 2, pieces, ,, no, abnormalities, <en...",7,train,"[649, 1, 2, 3, 4, 5, 650, 0, 0, 0, 0, 0, 0, 0,..."
1,GTEX-1117F-1026,Lung,"2 pieces, moderate congestion/moderate to mark...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, moderate, congestion/moderate, ...","[<start>, 2, pieces, ,, moderate, congestion/m...",10,train,"[649, 1, 2, 3, 6, 648, 7, 8, 9, 650, 0, 0, 0, ..."
2,GTEX-1117F-1126,Liver,"2 pieces, subtotal massive hepatic necrosis",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, subtotal, massive, hepatic, nec...","[<start>, 2, pieces, ,, subtotal, massive, hep...",9,train,"[649, 1, 2, 3, 648, 648, 10, 11, 650, 0, 0, 0,..."
3,GTEX-1117F-1226,Spleen,"2 pieces, marked congestion/autolysis",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, marked, congestion/autolysis]","[<start>, 2, pieces, ,, marked, congestion/aut...",7,train,"[649, 1, 2, 3, 8, 12, 650, 0, 0, 0, 0, 0, 0, 0..."
4,GTEX-1117F-1326,Adipose - Visceral (Omentum),"2 pieces, diffuse mesothelial hyperplasia; ~10...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, diffuse, mesothelial, hyperplas...","[<start>, 2, pieces, ,, diffuse, mesothelial, ...",16,train,"[649, 1, 2, 3, 13, 14, 15, 16, 17, 18, 648, 19..."


In [4]:
class PreLoadedReps(Dataset):

    def __init__(self, df_path, dtype='train', transform=None, target_transform=None):
        self.df_path = df_path
        df = pd.read_pickle(self.df_path)
        self.dtype=dtype
        self.df=df[df.dtype==dtype]
        self.transform = transform
        self.target_transform = target_transform
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        svs_path=self.df.iloc[idx]['svs_path']
        patch_path=self.df.iloc[idx]['patch_path']
        reps_path=self.df.iloc[idx]['reps_path']
        idx_tokens=self.df.iloc[idx]['idx_tokens']
        caplens=self.df.iloc[idx]['caplens']

        rep_tensors = torch.load(reps_path)
        caption_tensor = torch.LongTensor(idx_tokens)
        caplen = torch.LongTensor([caplens])
        
        if self.dtype=='train':
#             rep_tensor = rep_tensors[random.randint(0, len(rep_tensors)-1),:,:]
            rep_tensor=torch.mean(rep_tensors,dim=0)
            return rep_tensor.squeeze(0), caption_tensor, caplen
        else:
            rep_tensor=torch.mean(rep_tensors,dim=0)
            all_captions=torch.LongTensor([idx_tokens])
            return rep_tensor.squeeze(0), caption_tensor, caplen, all_captions

ds=PreLoadedReps(df_path,'val')

ds.__getitem__(0)[0].shape

torch.Size([192])

In [5]:
train_loader = torch.utils.data.DataLoader(
            PreLoadedReps(df_path,'val'),
            batch_size=32, shuffle=True, num_workers=1, pin_memory=True)